In [1]:
#!pip install sympy

In [2]:
import sympy as sp
sp.init_printing()

## Setup problem
An ideal gas compression happens from 1 bar to 2 bar at 298 K. Then an expansion occurs at constant 2 bar as the temperature rises to 373 K.
Let's setup the temperatures, pressures and volumes.

In [6]:
p,V,T=sp.symbols("p,V,T",real=True,positive=True)
R,Cv,Cp=sp.symbols("R,Cv,Cp",real=True,constant=True)
T1=298;T2=298;T3=373
p1=1;p2=2;p3=2
ig=sp.Eq(p*V,R*T) #ideal gas equation
V1=sp.solve(ig,V)[0].subs(p,p1).subs(T,T1).subs(R,0.08314)#solve for V and evaluate using p and T
V2=sp.solve(ig,V)[0].subs(p,p2).subs(T,T2).subs(R,0.08314)
V3=sp.solve(ig,V)[0].subs(p,p3).subs(T,T3).subs(R,0.08314)
print(V1,V2,V3)

24.7757200000000 12.3878600000000 15.5056100000000


## 1 - in terms of dT and dp
Let's calculate the heat using $$dq=C_p dT + l_p dp$$. The first step is isothermal so we only need to calculate using the pressure term. $$l_p=-T\left(\dfrac{\partial V}{\partial T}\right)$$

In [7]:
lp=-T*sp.diff(sp.solve(ig,V)[0],T)
q=sp.integrate(lp,(p,p1,p2))
q1=q.subs(R,8.314).subs(T,T1).evalf()
print("The heat in the isothermal step is ",q1.round(2)," J/mol")


The heat in the isothermal step is  -1717.32  J/mol


The second step is isobaric.

In [8]:
q=sp.integrate(Cp,(T,T2,T3))
q2=q.subs(Cp,20.785)
print("The heat in the isobaric step is ",q2.round(2), " J/mol")

The heat in the isobaric step is  1558.88  J/mol


In [9]:
q_tot1=q1+q2
print("The total heat for the two steps is ",q_tot1.round(2), " J/mol")

The total heat for the two steps is  -158.45  J/mol


## 2 - In terms of dT and dV
Let's calculate the same change using a different heat equation. $$dq=C_VdT+l_vdV$$

In [10]:
lv=T*sp.diff(sp.solve(ig,p)[0],T)
q=sp.integrate(lv,(V,V1,V2))
q1=q.subs(R,8.314).subs(T,T1).evalf()
print(q1)
print("The heat in the isothermal step is ",q1.round(2)," J/mol")


-1717.32204643426
The heat in the isothermal step is  -1717.32  J/mol


Because we didn't choose a natural equation to do the calculation, neither term is zero in the next step. We will have to do an integral for both terms.

In [11]:
q_a=sp.integrate(Cv,(T,T2,T3))
q_b=sp.integrate(p3,(V,V2,V3))*100 #100 changes pressure/volume energy to joules
q2=q_a.subs(Cv,12.471)+q_b

print("The heat in the isobaric step is ",q2.round(2), " J/mol")

The heat in the isobaric step is  1558.88  J/mol


In [12]:
q_tot2=q1+q2
print("The total heat for the two steps is ",q_tot2.round(2), " J/mol")
print("The total heat ",q_tot2.round(2)," should turn out to be identical to the heat using the other heat equation",q_tot1.round(2))

The total heat for the two steps is  -158.45  J/mol
The total heat  -158.45  should turn out to be identical to the heat using the other heat equation -158.45


## 3 - In terms of dp and dV
One more time with the last heat equation...  $ dq=\gamma_p dp + \gamma_V dV$. This is requires a bit more work (at least in the first step) because both p and V are changing. But we should get the same result.


In [13]:
gamv=Cp/sp.diff(sp.solve(ig,V)[0],T)
gamp=Cv/sp.diff(sp.solve(ig,p)[0],T)
q=sp.integrate(gamp.subs(V,sp.solve(ig,V)[0]),(p,p1,p2))+sp.integrate(gamv.subs(p,sp.solve(ig,p)[0]),(V,V1,V2)) #in order to get the right answer, we need to subsitute for p (or V) in terms of V (or p) before integrating.
q1=q.subs(R,0.08314).subs(T,T1).subs(Cp,20.785).subs(Cv,12.471).subs(p,p1).evalf()
print("The heat in the isothermal step is ",q1.round(2)," J/mol")


The heat in the isothermal step is  -1717.32  J/mol


For the isobaric case, the $dp$ term is zero but the integral in terms of V has both T and p in it. Convert $dV$ to $dT$ before integrating
$$dV=\left(\dfrac{\partial V}{\partial T}\right)dT$$

In [14]:
q=sp.integrate(gamv*sp.diff(sp.solve(ig,V)[0],T),(T,T2,T3)) #integrate gamma_v*(dV/dT)dT
q2=q.subs(p,sp.solve(ig,p)[0]).subs(Cp,20.785).evalf() #subsititute for p in terms of T (and V)
print("The heat in the isothermal step is ",q2.round(2)," J/mol")


The heat in the isothermal step is  1558.88  J/mol


In [15]:
q_tot2=q1+q2
print("The total heat for the two steps is ",q_tot2.round(2), " J/mol")
print("The total heat ",q_tot2.round(2)," should turn out to be identical to the heat using the other heat equations",q_tot1.round(2))

The total heat for the two steps is  -158.45  J/mol
The total heat  -158.45  should turn out to be identical to the heat using the other heat equations -158.45


## Work too!

In [16]:
w1=-sp.integrate(sp.solve(ig,p)[0],(V,V1,V2))
w2=-sp.integrate(p,(V,V2,V3))*100
w=(w1+w2).subs(p,p2).subs(T,298).subs(R,8.314)
print(w)
print(w+q_tot2)

1093.77204643427
935.325000000000


## Setup second problem
Let's change the order of the steps. First isobaric followed by isothermal. 
Now we'll setup the temperatures, pressures and volumes.

In [17]:
p,V,T=sp.symbols("p,V,T",real=True,positive=True)
R,Cv,Cp=sp.symbols("R,Cv,Cp",real=True,constant=True)
T1=298
T2=373
T3=373
p1=1
p2=1
p3=2
ig=sp.Eq(p*V,R*T)
V1=sp.solve(ig,V)[0].subs(p,p1).subs(T,T1).subs(R,0.08314)
V2=sp.solve(ig,V)[0].subs(p,p2).subs(T,T2).subs(R,0.08314)
V3=sp.solve(ig,V)[0].subs(p,p3).subs(T,T3).subs(R,0.08314)
print(V1,V2,V3)

24.7757200000000 31.0112200000000 15.5056100000000


## 1 - in terms of dT and dp
Let's calculate the heat using $$dq=C_p dT + l_p dp$$. The first step is isobaric so we only need to calculate using the pressure term. $$l_p=-T\left(\dfrac{\partial V}{\partial T}\right)$$

In [18]:
q=sp.integrate(Cp,(T,T1,T2))
q1=q.subs(Cp,20.785)
print("The heat in the isobaric step is ",q1.round(2), " J/mol")



The heat in the isobaric step is  1558.88  J/mol


The second step is isothermal.

In [19]:
lp=-T*sp.diff(sp.solve(ig,V)[0],T)
q=sp.integrate(lp,(p,p2,p3))
q2=q.subs(R,8.314).subs(T,T3).evalf()
print("The heat in the isothermal step is ",q2.round(2)," J/mol")

The heat in the isothermal step is  -2149.53  J/mol


In [20]:
q_tot2=q1+q2
print("The total heat for the two steps is ",q_tot2.round(2), " J/mol")

The total heat for the two steps is  -590.66  J/mol


In [26]:
w1=-sp.integrate(p,(V,V1,V2))*100
w2=-sp.integrate(sp.solve(ig,p)[0],(V,V2,V3))
w=(w1+w2).subs(p,p1).subs(T,373).subs(R,8.314)
print("Work is",w.round(2),"J/mol")
print("Work plus heat is ",(w+q_tot2).round(2),"J/mol")

Work is 1525.98 J/mol
Work plus heat is  935.32 J/mol


The two pathways have different heat and work but the same heat+work